<a href="https://colab.research.google.com/github/oksami/cs6320project/blob/main/BERT_Model_%26_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Load data
data = pd.read_csv("/content/drive/MyDrive/Data_NLP/intents_combined.csv")

print(data.head())


                                               Query     Intent
0               Can you recommend fossil-free funds?  Recommend
1                Suggest some gender equality funds.  Recommend
2  What are the best deforestation-free investments?  Recommend
3                Show me some funds free of weapons.  Recommend
4      I need a list of gun-free investment options.  Recommend


In [ ]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
import torch

# loading the labelled dataset
df = pd.read_csv("intents_combined.csv")

# Convert the dataset into a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

#Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the queries
def tokenize_function(examples):
    return tokenizer(examples['Query'], padding=True, truncation=True)

# Apply tokenization to the dataset
dataset = dataset.map(tokenize_function, batched=True)

# Map the Intent column to numeric labels
intent_labels = {label: idx for idx, label in enumerate(df['Intent'].unique())}
dataset = dataset.map(lambda x: {'labels': intent_labels[x['Intent']]})

# Convert to pandas DataFrame for splitting
df = pd.DataFrame(dataset)

# Split dataset into train and test using pandas
train_df, test_df = train_test_split(df, test_size=0.2)

# Convert the split DataFrames back to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Define the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(intent_labels))

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=3,              # Number of epochs
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    evaluation_strategy="epoch",     # Evaluation at the end of each epoch
    save_strategy="epoch",           # Save model after each epoch
    logging_dir='./logs',            # Directory for logs
    logging_steps=10,
    load_best_model_at_end=True,     # Load best model at the end of training
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,1.033643
2,1.117600,0.958645
3,0.937000,0.894640


TrainOutput(global_step=27, training_loss=0.962304362544307, metrics={'train_runtime': 219.4174, 'train_samples_per_second': 0.984, 'train_steps_per_second': 0.123, 'total_flos': 3108027246912.0, 'train_loss': 0.962304362544307, 'epoch': 3.0})

In [ ]:
eval_results = trainer.evaluate(test_dataset)
print(eval_results)


{'eval_loss': 0.8946395516395569, 'eval_runtime': 2.8257, 'eval_samples_per_second': 6.37, 'eval_steps_per_second': 1.062, 'epoch': 3.0}


In [ ]:
# Push the model to Hugging Face to avoid retraining every time
model.save_pretrained("./bert-intent-classifier")
tokenizer.save_pretrained("./bert-intent-classifier")



('./bert-intent-classifier/tokenizer_config.json',
 './bert-intent-classifier/special_tokens_map.json',
 './bert-intent-classifier/vocab.txt',
 './bert-intent-classifier/added_tokens.json')

In [ ]:
from huggingface_hub import login

login(token="")  # Replace with your token from Hugging face


In [ ]:
from huggingface_hub import HfApi

# Push the model
model.push_to_hub("bert-intent-classifier")

# Push the tokenizer
tokenizer.push_to_hub("bert-intent-classifier")


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/appledewberry/bert-intent-classifier/commit/b43a7d26d51052175be2da5e14410ad8120b92b7', commit_message='Upload tokenizer', commit_description='', oid='b43a7d26d51052175be2da5e14410ad8120b92b7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/appledewberry/bert-intent-classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='appledewberry/bert-intent-classifier'), pr_revision=None, pr_num=None)

In [18]:
# Load the model and tokenizer from Hugging Face
from transformers import BertForSequenceClassification, BertTokenizer

model = BertForSequenceClassification.from_pretrained("appledewberry/bert-intent-classifier")
tokenizer = BertTokenizer.from_pretrained("appledewberry/bert-intent-classifier")


In [ ]:
pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [21]:
import torch
import re
import pandas as pd
from datasets import Dataset

# load the labelled intents
df = pd.read_csv("/content/drive/MyDrive/Data_NLP/intents_combined.csv")
# Define the data as a Hugging face dataset
dataset = Dataset.from_pandas(df)
# Process the labelled intents
intent_labels = {label: idx for idx, label in enumerate(df['Intent'].unique())}
reverse_intent_labels = {v: k for k, v in intent_labels.items()}
# Function that will process query - extract the intent, criterion and grades from the query
def process_query(query, tokenizer, model, reverse_intent_labels):
    """
    Processes the query to recognize intent and extract relevant entities (fund type, grades).

    Args:
        query (str): The user's input query.
        tokenizer: The tokenizer used for the model.
        model: The trained model for intent recognition.
        reverse_intent_labels (dict): Mapping of intent IDs to intent labels.

    Returns:
        dict: A dictionary containing the predicted intent, fund type, and grades.
    """
    # Tokenize the query
    inputs = tokenizer(query, return_tensors="pt", padding=True, truncation=True, max_length=128)

    # Run the model to get the logits
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the predicted label
    logits = outputs.logits
    pred_label = logits.argmax(dim=-1).item()
    predicted_intent = reverse_intent_labels.get(pred_label, "Unknown Intent")


    # Define regex patterns for fund type and grades
    fund_type_pattern = r"(fossil-free|gender equality|deforestation-free|gun-free|tobacco-free|prison-free)"
    grade_pattern = r"\b[A-F]\b"

    # Extract fund type
    fund_type_match = re.search(fund_type_pattern, query)
    fund_type = fund_type_match.group(0) if fund_type_match else "Not specified"

    # Extract grades
    grades = re.findall(grade_pattern, query)
    grade = ', '.join(grades) if grades else "Not specified"

    # Combine results into a dictionary
    result = {
        "intent": predicted_intent,
        "fund_type": fund_type,
        "grades": grade
    }

    return result

# Example
if __name__ == "__main__":

    query = "Can you Recommend fossil-free and prison-free with C and B grade?"
    result = process_query(query, tokenizer, model, reverse_intent_labels)

    print(f"Intent: {result['intent']}")
    print(f"Fund Type: {result['fund_type']}")
    print(f"Grades: {result['grades']}")




Intent: List
Fund Type: fossil-free
Grades: C, B


In [ ]:
import pandas as pd
import os
# Function that will extract the relevant funds based on the criterions and grades obtained
def fetch_data(processed_query, data_directory="data"):
    """
    Fetches data from the relevant CSV file based on the processed query.

    Args:
        processed_query (dict): Output from the process_query function containing 'intent', 'fund_type', and 'grades'.
        data_directory (str): Directory where the CSV files are stored.

    Returns:
        pd.DataFrame or str: A filtered DataFrame based on the query or a message if no matching data is found.
    """
    # Extract fund type and grades from the processed query
    fund_type = processed_query.get("fund_type", "Not specified").replace("-", "").lower()
    grades = processed_query.get("grades", "Not specified").split(", ")

    # Map fund type to CSV file
    file_name = f"{fund_type}.csv"
    file_path = os.path.join(data_directory, file_name)

    # Check if the file exists
    if not os.path.exists(file_path):
        return f"No data available for fund type: {fund_type}."

    # Load the CSV file
    df = pd.read_csv(file_path)

    # If grades are specified, filter the data
    if grades != ["Not specified"]:
        df = df[df['Grade'].isin(grades)]

    # If no data matches, return a message
    if df.empty:
        return f"No matching data found for {fund_type} funds with grades: {', '.join(grades)}."

    return df

# Example Usage
if __name__ == "__main__":
    # Assuming process_query and its dependencies are already defined
    query = "Show me deforestation-free funds with grade A."
    processed_query = process_query(query, tokenizer, model, reverse_intent_labels)

    # Fetch the relevant data
    result = fetch_data(processed_query, data_directory="/content/drive/MyDrive/Data_NLP")

    # Output result
    if isinstance(result, pd.DataFrame):
        print("Filtered Data:")
        print(result)
    else:
        print(result)


Filtered Data:
                                Fund profile: Fund name Fund profile: Ticker  \
0                                  ALPS Active REIT ETF                 REIT   
1                                 ALPS Clean Energy ETF                 ACES   
2                        ALPS Medical Breakthroughs ETF                 SBIO   
3                           ALPS REIT Dividend Dogs ETF                 RDOG   
4                     Alerian Energy Infrastructure ETF                 ENFR   
...                                                 ...                  ...   
1947  WisdomTree Efficient Gold Plus Gold Miners Str...                 GDMN   
1948             WisdomTree New Economy Real Estate ETF                 WTRE   
1949          Xtrackers Cybersecurity Select Equity ETF                 PSWD   
1950          Xtrackers Semiconductor Select Equity ETF                 CHPS   
1951  Xtrackers US Green Infrastructure Select Equit...                 UPGR   

     Fund profile: Share

In [ ]:
import pandas as pd
import os

def fetch_data(processed_query, data_directory="data"):
    """
    Fetches data from the relevant CSV file based on the processed query
    and generates a chatbot-friendly response.

    Args:
        processed_query (dict): Output from the process_query function containing 'intent', 'fund_type', and 'grades'.
        data_directory (str): Directory where the CSV files are stored.

    Returns:
        str: A chatbot-friendly response with the top recommendations.
    """
    # Extract fund type and grades from the processed query
    fund_type = processed_query.get("fund_type", "Not specified").replace("-", "").lower()
    grades = processed_query.get("grades", "Not specified").split(", ")

    # Map fund type to CSV file
    file_name = f"{fund_type}.csv"
    file_path = os.path.join(data_directory, file_name)

    # Check if the file exists
    if not os.path.exists(file_path):
        return f"No data available for fund type: {fund_type}."

    # Load the CSV file
    df = pd.read_csv(file_path)

    # If grades are specified, filter the data
    if grades != ["Not specified"]:
        df = df[df['Grade'].isin(grades)]

    # If no data matches, return a message
    if df.empty:
        return f"No matching data found for {fund_type} funds with grades: {(grades)}."

    # Create a chatbot-friendly response
    top_recommendations = df.head(5)  # Limit to top 5 results
    response = f"Here are the top 5 recommendations for {fund_type.replace('-', ' ')} funds with grades: {(grades)}:\n"

    for index, row in top_recommendations.iterrows():
        response += f"\nFund: {row['Fund profile: Fund name']} ({row['Fund profile: Ticker']})\n"
        response += f"  - Grade: {row['Grade']}\n"
        response += f"  - Share Class: {row['Fund profile: Shareclass type']}\n"
        response += f"  - Category: {row['Fund profile: Category group']}\n"
        response += f"  - Weight: {row['Weight']}\n"

    return response

# Example Usage
if __name__ == "__main__":

    query = "Show me fossil-free funds with grade A."
    processed_query = process_query(query, tokenizer, model, reverse_intent_labels)

    # Fetch the relevant data and generate a chatbot-friendly response
    chatbot_response = fetch_data(processed_query, data_directory="/content/drive/MyDrive/Data_NLP")
    print(chatbot_response)


Here are the top 5 recommendations for fossilfree funds with grades: A:

Fund: ALPS Active REIT ETF (REIT)
  - Grade: A
  - Share Class: ETF
  - Category: Sector Equity Funds
  - Weight: 0.0

Fund: ALPS Global Travel Beneficiaries ETF (JRNY)
  - Grade: A
  - Share Class: ETF
  - Category: Sector Equity Funds
  - Weight: 0.0

Fund: ALPS Medical Breakthroughs ETF (SBIO)
  - Grade: A
  - Share Class: ETF
  - Category: Sector Equity Funds
  - Weight: 0.0

Fund: ALPS O'Shares Global Internet Giants ETF Shares (OGIG)
  - Grade: A
  - Share Class: ETF
  - Category: Sector Equity Funds
  - Weight: 0.0

Fund: ALPS REIT Dividend Dogs ETF (RDOG)
  - Grade: A
  - Share Class: ETF
  - Category: Sector Equity Funds
  - Weight: 0.0

